In [2]:
!curl https://raw.githubusercontent.com/vseloved/prj-nlp-2019/master/tasks/07-language-as-sequence/run-on-test.json --output run-on-test.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  273k  100  273k    0     0   492k      0 --:--:-- --:--:-- --:--:--  492k


In [2]:
import json

with open('run-on-test.json') as f:
    run_on_js = json.load(f)

In [3]:
import random

def rand_run_on_num():
    x = random.random()
    if x < 0.25:
        return 0
    elif 0.25 <= x < 0.97:
        return 1
    
    return 2

In [4]:
from nltk.corpus import brown

In [5]:
from collections import defaultdict
import nltk

In [11]:
random.seed(273757)

bigrams = defaultdict(int)
trigrams = defaultdict(int)
corpus = []

def tag_tokens(tokens):
    
    tagged = [[token, False] for token in tokens]
    
    if tagged[-1][0][-1] in '.!?':
        tagged[-1] = ['.', False]
    else:
        tagged.append(['.', False])        
    
    return tagged

with open('run-on-corpus.txt', 'w') as f_corpus:
    for fileid in brown.fileids():
        sents = brown.sents(fileid)
        run_on_num = rand_run_on_num()
        buff = []
        prev_offset = 0
        file_tokens = []
        for sent in sents[1:]:
            if len(sent) < 2:
                continue
                
            tokens = tag_tokens(sent)
            buff.extend(tokens)
            file_tokens.extend(map(lambda t: t[0],tokens))
                
            if prev_offset > 0:                
                if buff[prev_offset - 1][0] == '.':
                    del buff[prev_offset - 1]
                    buff[prev_offset - 2][1] = True                
                   
                    if not buff[prev_offset - 1][0].isupper():                
                        lwr = random.random()                
                        if lwr < 0.93:
                            buff[prev_offset - 1][0] = buff[prev_offset -1][0].lower()                                            
            
            prev_offset = len(buff)            
            
            if run_on_num == 0:
                print(buff, file = f_corpus, end = "\n")
                corpus.append(buff)
                run_on_num = rand_run_on_num()
                prev_offset = 0
                buff = []                               
                       
            run_on_num -= 1
     
        for g in nltk.ngrams(file_tokens, 2):        
            bigrams[g] += 1
                
        for g in nltk.ngrams(file_tokens, 3):    
            trigrams[g] += 1    

In [7]:
def update_freq(ngrams):
    n = sum(ngrams.values())
    for k, v in ngrams.items():
        ngrams[k] = v/n

In [12]:
update_freq(bigrams)
update_freq(trigrams)

In [9]:
sorted(bigrams.items(), key=lambda kv: kv[1], reverse = True)

[(('of', 'the'), 0.008234774245535599),
 (('.', 'The'), 0.005543569615207992),
 ((',', 'and'), 0.005411868680304645),
 (('in', 'the'), 0.0047481652846733035),
 (('.', '``'), 0.0035602575099858767),
 ((',', 'the'), 0.003211943195307288),
 (('to', 'the'), 0.002934678069194978),
 (("''", '.'), 0.0029017528354691414),
 (('.', 'He'), 0.0024615944477658497),
 ((';', ';'), 0.002394877526795075),
 ((';', '.'), 0.002394877526795075),
 (('?', '.'), 0.002022302513581659),
 (('on', 'the'), 0.0019720482094738026),
 (('and', 'the'), 0.0018290833788221432),
 (("''", ','), 0.0017485032015457532),
 (('.', 'It'), 0.0016393300581390312),
 ((',', 'but'), 0.0015960073821839828),
 (('for', 'the'), 0.001507629123235684),
 (('to', 'be'), 0.0014599741796851309),
 (('.', 'In'), 0.0014461109233795152),
 (('at', 'the'), 0.0012979473716132498),
 ((',', 'he'), 0.0012858170223458362),
 (('with', 'the'), 0.0012632892308492112),
 (('of', 'a'), 0.0012494259745435955),
 (('that', 'the'), 0.0011749109719009123),
 (('.', 

In [18]:
train_index = int(0.7 * len(corpus))

train_data = corpus[: train_index]
test_data = corpus[train_index: ]

In [31]:
def baseline(data):
    result_data = []
    for sent in data:
        result_sent = []
        for i, word in enumerate(sent):
            if i > 2 and i < (len(sent) - 1):
                pbigram = (word[0], '.')
                bigram = (word[0], sent[i+1][0])
                if bigrams[pbigram] !=0 and bigrams[bigram] != 0:
                    if bigrams[pbigram] > bigrams[bigram]:
                        result_sent.append([word[0], True])
                    else:
                        result_sent.append(word)
                else:
                    result_sent.append(word)
            else:
                result_sent.append(word)
            
        result_data.append(result_sent)   
    
    return result_data

In [34]:
result = baseline(train_data)

In [35]:
def labels_vec(data):
    return [word[1] for sent in data for word in sent]

In [37]:
from sklearn.metrics import classification_report


print(classification_report(labels_vec(train_data), labels_vec(result)))

              precision    recall  f1-score   support

       False       1.00      0.66      0.79     44268
        True       0.02      1.00      0.04       323

   micro avg       0.66      0.66      0.66     44591
   macro avg       0.51      0.83      0.42     44591
weighted avg       0.99      0.66      0.79     44591

